In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import mdtraj as md
from tqdm import tqdm

In [ ]:
aa_total_traj = None
for i in tqdm(range(557)):
    if i < 10:
        stri = '00' + str(i)
    elif i < 100:
        stri = '0' + str(i)
    else:
        stri = str(i)
    traj = md.load(f'/project/andrewferguson/DESRES/DESRES-Trajectory_NTL9-0-protein/NTL9-0-protein/NTL9-0-protein-{stri}.dcd', top='/project/andrewferguson/DESRES/DESRES-Trajectory_NTL9-0-protein/system_reorder.pdb', stride=100)
    if not i:
        aa_total_traj = traj
    else:
        aa_total_traj = aa_total_traj.join(traj)

In [ ]:
aa_total_traj.n_frames

In [ ]:
for j, frame in tqdm(enumerate(aa_total_traj[::27])):
    frame.atom_slice(frame.top.select('not element H and not name OXT')).save_pdb(f'data/NTL9/frame_{j}.pdb')

In [ ]:
def leader_clustering(traj, K):
    """
    Perform leader clustering on an MD trajectory with an RMSD cutoff K.
    
    Parameters:
        traj (mdtraj.Trajectory): MD trajectory (only alpha carbons).
        K (float): RMSD cutoff in Angstroms.

    Returns:
        cluster_labels (list): Cluster assignments for each frame.
    """
    cluster_representatives = []  # Store one representative frame per cluster
    cluster_labels = np.zeros(traj.n_frames, dtype=int)  # Cluster labels
    cluster_count = 0  # Number of clusters
    all_frames = []
    for i in tqdm(range(traj.n_frames)):
        frame = traj[i]
        assigned = False

        # Compare against existing cluster representatives
        for j, rep in enumerate(cluster_representatives):
            rmsd = md.rmsd(frame, rep)[0]  # Compute RMSD to representative frame
            if rmsd < K:
                cluster_labels[i] = j + 1  # Assign to cluster
                assigned = True
                break

        # If no cluster found, create a new one
        if not assigned:
            cluster_count += 1
            cluster_labels[i] = cluster_count
            cluster_representatives.append(frame)
            all_frames.append(i)

    return cluster_labels, cluster_representatives, all_frames

In [ ]:
K = 0.2 # Angstroms

# Perform clustering
clusters, reps, frames = leader_clustering(total_traj, K)

# Print results
print(f"Total clusters: {len(set(clusters))}")

In [ ]:
all_reps = reps[0]
for rep in reps:
    rep.superpose(reps[0])
    all_reps = all_reps.join(rep)

In [ ]:
for i in range(1, len(set(clusters)) + 1):
    cur_cluster_idxs = np.where(clusters == i)[0]
    # print(aa_total_traj[cur_cluster_idxs])
    aa_total_traj[cur_cluster_idxs].save_xtc(f'data/clusters/{i-1}/chignolin_{i-1}.xtc')
    

In [ ]:
from tqdm import tqdm
pdb_energies = []
plt.hist(energies, 30)
plt.xlabel('Energies (kJ/mol)')
plt.ylabel('Frequency (kJ/mol)')
for i in tqdm(range(1, 201)):
    pdb_energies.append(compute_pdb_energy(f'outputs/clusters/0/Pro_pretrained_ckp-14_noise-0.003_chi-fix-0.2_masked/ca_rep_{i}.pdb'))

plt.hist(pdb_energies, 30)


In [ ]:
import mdtraj as md
import nglview as nv

In [ ]:
import subprocess
import openbabel
import warnings
import pybel 
%load_ext autoreload 
%autoreload 2
from rdkit import Chem
from energy_utils import charmm_structure_to_energy
import multiprocessing as mp
topology_file = "/project/andrewferguson/DESRES/DESRES-Trajectory_CLN025-0-protein/system_reorder.pdb"
warnings.filterwarnings('ignore')
# for n in tqdm(range(444)):
#     energies = []
#     traj = md.load(f"data/clusters/{n}/chignolin_{n}.xtc", top=topology_file)
#     traj = traj.atom_slice(traj.top.select('not element H and not name OXT'))
#     for i in range(len(traj)):
#         try:
#             energy, _ = charmm_structure_to_energy(traj.top, traj[i].xyz)
#             energies.append(energy)
#         except e:
#             print(f'failed at {i}: {e}')
#         if not i % 300:
#             energies_np = np.array(energies)
#             np.save(f"data/clusters/{n}/energies.npy", energies_np)
#     energies_np = np.array(energies)
#     np.save(f"data/clusters/{n}/energies.npy", energies_np)


warnings.filterwarnings('ignore')

def compute_energy(frame_idx, traj):
    try:
        energy, _ = charmm_structure_to_energy(traj.top, traj[frame_idx].xyz)
        return frame_idx, energy
    except Exception as e:
        print(f'Failed at frame {frame_idx}: {e}')
        return frame_idx, None

def process_cluster(n):
    energies = []
    traj = md.load(f"data/clusters/{n}/chignolin_{n}.xtc", top=topology_file)
    traj = traj.atom_slice(traj.top.select('not element H and not name OXT'))
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(compute_energy, [(i, traj) for i in range(len(traj))])
    
    results.sort()  # Ensure results are in order
    energies = [energy for _, energy in results if energy is not None]
    
    energies_np = np.array(energies)
    np.save(f"data/clusters/{n}/energies.npy", energies_np)

for n in tqdm(range(1)):
    process_cluster(n)

In [ ]:
# from rdkit import Chem
# from rdkit.Chem import AllChem
# from rdkit import rdBase
# import warnings
%load_ext autoreload 
%autoreload 2
import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)
from tqdm import tqdm
# blocker = 'hi'
# del blocker
# energies = np.load('data/clusters/0/energies.npy')
# plt.hist(energies, 20, label='Unbiased Trajectory')
name_dict = {
    'GTT': 'WW Domain',
    'PRB': 'Protein B',
    'chignolin': 'Chignolin',
    '2JOF': 'Trp-Cage'
}
# blocker = rdBase.BlockLogs()
np_bins = np.linspace(-4000, 8000, 100)
for protein in ['GTT', 'PRB', 'chignolin', '2JOF']:
    energies = np.load(f"energy_files/energies_{protein}_nomodel.npy")
    flowback_energies = np.load(f"energy_files/energies_{protein}_sc2_ckp-250.npy")
    flowback_euler_energies = np.load(f"energy_files/energies_{protein}_sc2_ckp-250_euler.npy")
    hist, bins = np.histogram(flowback_euler_energies, np_bins, density=True)
    plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='red', label='Original Flowback')
    # gtt_energies = np.load(f"energy_files/energies_GTT_gtt_post_ckp-2.npy")
    hist, bins = np.histogram(flowback_energies, np_bins, density=True)
    plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='blue', label='Flowback + LJ/bonds')
    try:
        flowback_adj_energies = np.load(f"energy_files/energies_{protein}_acc_grad_16_post_ckp-1.npy")
        gtt_energies = np.load(f"energy_files/energies_{protein}_gtt_16_post_ckp-7.npy")
        # flowback_adj_energies = np.load(f"energy_files/energies_{protein}_nb2_post_ckp-0.npy")
        hist, bins = np.histogram(flowback_adj_energies, np_bins, density=True)
        plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='green',  label='Flowback Adjoint + LJ/bonds')
        hist, bins = np.histogram(gtt_energies, np_bins, density=True)
        plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='purple',  label='Flowback GTT')
    except:
        pass
    hist, bins = np.histogram(energies, np_bins, density=True)
    plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='black',  label='Unbiased Distribution')
    # hist, bins = np.histogram(gtt_energies, np.linspace(-5000, 25000, 100))
    # plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='red',  label='GTT-only adjoint')
    plt.xlabel('U (kJ/mol)')
    plt.ylabel('Frequency')
    plt.title(f'{name_dict.get(protein, protein)}')
    plt.legend(fontsize=12)
    plt.show()



In [ ]:
energies = np.load(f"energy_files/energies_short_pdbs_nomodel.npy")
with open('a16.out', 'r') as f:
    lines = f.readlines()
def is_float(element: any) -> bool:
    #If you expect None to be passed:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False

energies_early = []
energies_late = []
for line in lines[:400]:
    l = line.strip()
    if not l.isdigit() and is_float(l):
        energies_early.append(float(l))
for line in lines[6000:] :
    l = line.strip()
    if not l.isdigit() and is_float(l):
        energies_late.append(float(l))

hist, bins = np.histogram(energies_early, np.linspace(-8000, 14000, 25), density=True)
plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='blue',  label='Flowback')

hist, bins = np.histogram(energies_late, np.linspace(-8000, 14000, 25), density=True)
plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='green',  label='Flowback Adjoint')

hist, bins = np.histogram(energies, np.linspace(-8000, 12000, 25), density=True)
plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, c='black',  label='True Energies')

plt.legend()

In [ ]:
from scipy.stats import gmean
min_val = np.min(energies)
gmean(energies - min_val + 1) + min_val - 1

In [ ]:
import nglview as nv
view = nv.show_file(f'jobs/post_train_world/temp.pdb')
view.clear()
view.add_ball_and_stick()
view

In [ ]:
from rdkit import Chem
mol = Chem.MolFromPDBFile(f'jobs/post_train_world/temp.pdb')

In [ ]:
topology_file = "/project/andrewferguson/DESRES/DESRES-Trajectory_CLN025-0-protein/system.pdb"
from tqdm import tqdm
import mdtraj as md
import os
os.makedirs(f'data/clusters/all_aa', exist_ok=True)
for i in tqdm(range(1)):
    traj = md.load(f'data/clusters/{i}/chignolin_{i}.xtc', top=topology_file)
    # ca_traj = traj.atom_slice(traj.top.select('name CA'))
    traj[0].save_pdb(f'data/clusters/all_aa/{i}_rep.pdb')

In [ ]:
%load_ext autoreload
%autoreload 2
pdb = md.load('data/finetune_chignolin/frame_0.pdb')
# pdb = pdb.atom_slice(pdb.top.select('not element H and not name OXT'))

In [ ]:
import pickle
pdb.xyz[:, :10], pickle.load(open('train_features/feats_chignolin.pkl', 'rb+'))['xyz'][0][:10]

In [ ]:
features = pickle.load(open('train_features/feats_chignolin.pkl', 'rb+'))

In [7]:
%load_ext autoreload
%autoreload 2
import nglview as nv 
import mdtraj as md
from utils import bond_fraction
traj = md.load('outputs/one_GTT/n010_post_ckp-2000/frame_0_dt.pdb')[-1]
traj2 = md.load('data/one_GTT_clean_AA/frame_0.pdb')
traj2.superpose(traj)
print(bond_fraction(traj, traj2))
traj.center_coordinates()
view = nv.show_mdtraj(traj[::-1])
view.clear()
view.add_ball_and_stick()
a = view.add_trajectory(traj2)
a.clear_representations()
a.add_representation('ball+stick', opacity=0.3)
# view.add_unitcell()
view

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.911864406779661


NGLWidget()

In [ ]:
np.load('jobs/hi_charmm_post/losses-epoch-3.npy')

In [ ]:
 np.load(f"energy_files/energies_{protein}_acc_grad_post_16_post_ckp-1.npy")

In [9]:
traj = md.load('outputs/one_GTT/big_model_ckp-15/frame_0_dt.pdb')
# traj2 = md.load('data/one_GTT_clean_AA/frame_0.pdb')
# traj2.superpose(traj)
# print(bond_fraction(traj, traj2))
traj.center_coordinates()
view = nv.show_mdtraj(traj[::-1])
view.clear()
view.add_ball_and_stick()

# view.add_unitcell()
view

NGLWidget(max_frame=99)